In [ ]:
%matplotlib nbagg
import numpy as np
import matplotlib.pyplot as plt
from sympy import *
import sympy.physics.mechanics as me
#from sympy import sin, cos, symbols, solve, Matrix
#from scipy.integrate import odeint
from IPython.display import SVG
import time
me.init_vprinting()#(use_latex='mathjax')
#from sympy.interactive import printing
#printing.init_printing(use_latex=True)

In [ ]:
def mprint(d):
    print(me.vlatex(d))
    return d
    

In [ ]:
SVG(filename='RovVision2v1.svg')

# Define Variables

constants:

- $m_A$: Mass of the Rov.
- $a,b,c,d,e,f,g,h,i$: reltive coords wrt COM

In [ ]:
# Inertial Reference Frame
N = me.ReferenceFrame('N')

# Define a world coordinate origin
O = me.Point('O')
O.set_vel(N, 0)

In [ ]:
#generelized coordinates
#q0..3 = xyz positions  q4..6 = yaw,pitch,roll rotations
q = list(me.dynamicsymbols('q0:6'))
#generlized speeds
u = list(me.dynamicsymbols('u0:6'))

kin_diff=Matrix(q).diff()-Matrix(u)
#print(me.vlatex(kin_diff))
mprint(kin_diff)

In [ ]:
# Constants for the ROV Body

m_b = symbols('m_b')                                 # Mass of the body
v_b = symbols('v_b') # Volume of the body
mux = symbols('\mu_x') #drag
muy = symbols('\mu_y') #drag
muz = symbols('\mu_z') #drag
mu_r = symbols('\mu_r') #rotational drag
g = symbols('g')
I = list(symbols('Ixx, Iyy, Izz'))             # Moments of inertia of body
a,b,c,d,e,f,h,i,j = symbols(list('abcdefhij'))

In [ ]:
# Robot Reference Frame
Rz=N.orientnew('R_z', 'Axis', (q[3+0], N.z))
Rz.set_ang_vel(N,u[3+0]*N.z)

Ry=Rz.orientnew('R_y', 'Axis', (q[3+1], Rz.y))
Ry.set_ang_vel(Rz,u[3+1]*Rz.y)

R=Ry.orientnew('R', 'Axis', (q[3+2], Ry.x))
R.set_ang_vel(Ry,u[3+2]*Ry.x)


# Center of mass of body
COM = O.locatenew('COM', q[0]*N.x + q[1]*N.y + q[2]*N.z)

# Set the velocity of COM
COM.set_vel(N, u[0]*N.x + u[1]*N.y + u[2]*N.z)

#center of bouyency
COB = COM.locatenew('COB', -R.z*e)
COB.v2pt_theory(COM, N, R);
COD = COM.locatenew('COD', -R.z*f-R.x*a)
COD.v2pt_theory(COM, N, R);


In [ ]:
#mprint(COB.pos_from(COM))

In [ ]:
# Calculate inertia of body
#for the products of inertia we assume symetric body => ixy=0, iyz=0, izx=0
Ib = me.inertia(R, *I , ixy=0, iyz=0, izx=0)
# Create a rigid body object for body
Body = me.RigidBody('Body', COM, R, m_b, (Ib, COM))
Ib

In [ ]:
# Points of thrusters
L1 = COM.locatenew('L_1', -R.y*d+R.x*i-R.z*h)
L2 = COM.locatenew('L_2', R.y*d+R.x*i-R.z*h)
L3 = COM.locatenew('L_3', R.y*d-R.x*i-R.z*h)
L4 = COM.locatenew('L_4', -R.y*d-R.x*i-R.z*h)
L5 = COM.locatenew('L_5', -R.y*c +R.x*b+R.z*j)
L6 = COM.locatenew('L_6',  R.y*c +R.x*b+R.z*j)
L7 = COM.locatenew('L_7', R.y*c-R.x*b+R.z*j)
L8 = COM.locatenew('L_8', -R.y*c-R.x*b+R.z*j)

# Set the velocity of points 
L1.v2pt_theory(COM, N, R)
L2.v2pt_theory(COM, N, R)
L3.v2pt_theory(COM, N, R);
L4.v2pt_theory(COM, N, R);
L5.v2pt_theory(COM, N, R);
L6.v2pt_theory(COM, N, R);
L7.v2pt_theory(COM, N, R);
L8.v2pt_theory(COM, N, R);


## Calculating hydrodynamic drag

https://en.wikipedia.org/wiki/Drag_(physics)

for slow moving objects with non turbulant flow the drag is proportional to the velocity and can be written as:

$F_{D}\,=\mu \,v$

In [ ]:
#v=N.x*u[0]+N.y*u[1]+N.z*u[2]
Fd=-u[0]*mux*R.x -u[1]*R.y*muy -u[2]*muz*R.z
Fd

In [ ]:
#thrust forces symbols
F1, F2, F3, F4, F5, F6, F7, F8 = symbols('f_1, f_2, f_3, f_4, f_5, f_6, f_7, f_8') 
Fg = -N.z *m_b * g
Fb = N.z * v_b * 1e3 *g #whight of 1m^3 water in kg (MKS units)
mprint(Fb)

In [ ]:
#### adding dumping Torqe for each rotation

#T_z=(Rz,-u[3+2]*N.z*mu_r) #rotaional dumping Torqe
#T_y=(Ry,-u[3+1]*Rz.y*mu_r) #rotaional dumping Torqe
#T_x=(R,-u[3+0]*Ry.x*mu_r) #rotaional dumping Torqe
#T = (R, -(u[3+0]*Ry.x+u[3+1]*Rz.y+u[3+2]*N.z)*mu_r)
T = (R, -(u[3+0]*R.z+u[3+1]*R.y+u[3+2]*R.x)*mu_r)
mprint(T[1])

In [ ]:
kane = me.KanesMethod(N, q_ind=q, u_ind=u, kd_eqs=kin_diff)

In [ ]:
sq2=0.7#np.sqrt(2)/2
bodies = (Body,)
loads = (
    (L1, F1 * R.z),
    (L2, F2 * R.z),
    (L3, F3 * R.z), 
    (L4, F4 * R.z), 
    (L5, sq2 * F5 * (R.x + R.y)), 
    (L6, sq2 * F6 * (R.x - R.y)), 
    (L7, sq2 * F7 * (R.x + R.y)), 
    (L8, sq2 * F8 * (R.x - R.y)), 
    (COM, Fg ), 
    (COB, Fb ), 
    (COD, Fd ),
    T
    )

fr, frstar = kane.kanes_equations(loads=loads, bodies=bodies)

In [ ]:
#u_dot=trigsimp(trigsimp(kane.mass_matrix).inv()*trigsimp(kane.forcing))
u_dot=kane.mass_matrix.inv()*kane.forcing
u_dot.atoms(Symbol)

In [ ]:
mprint(F1 * R.x)

In [ ]:
if 0:
    km_simp=trigsimp(kane.mass_matrix)
    kf_simp=trigsimp(kane.forcing)
#mprint(kf_simp)

In [ ]:
subs=[(a,0.0),
    (b,0.20),
    (c,0.20), #c & b should be the sae value to achive 45 deg from COM
    (d,0.15),
    (e,0.00),
    (f,0.00),
    (h,0.00),
    (i,0.20),
    (j,0.00),
    (m_b,1.0), #1
    (v_b,0.001), #0.001
    (mux,0.2),
    (muy,0.2),
    (muz,0.2),
    (mu_r,0.3),
    (g,9.8), #9.8
    (I[0],0.5),
    (I[1],0.5),
    (I[2],0.5)
     ]#mprint(kf_simp[4])
#u_dot_simp[1]
u_dot_simp=u_dot.subs(subs)
#u_dot_simp=trigsimp(u_dot_simp)


In [ ]:
#dump
#import dill
#dill.settings['recurse'] = True
#dill.dump(u_dot,open('udot.pkl','wb'))

In [ ]:
u_dot.atoms(Symbol)

In [ ]:
if 0:
    def get_next_state_slow(curr_q,curr_u,control,curr_t,dt,subs):
        #for i in range(len(q_vec)):
        rg=range(len(curr_q))
        subsqf=[(q[i],curr_q[i]) for i in rg] + [(u[i],curr_u[i]) for i in rg]+subs
        forces=control(curr_t)
        subsqf+=[(F1,forces[0]),
                 (F2,forces[1]),
                 (F3,forces[2]),
                 (F4,forces[3]),
                 (F5,forces[4]),
                 (F6,forces[5]),
                 (F7,forces[6]),
                 (F8,forces[7])]
        if 0:
            u_dot_f=u_dot_simp.subs(subsqf).evalf()
        else:
            km=km_simp.subs(subsqf).evalf()
            kf=kf_simp.subs(subsqf).evalf()
            u_dot_f=km.inv()*kf
        #print('--',u_dot_f)
        u_dot_f=np.array(u_dot_f).flatten()
        next_q=curr_q+curr_u*dt
        next_u=curr_u+u_dot_f*dt
        return next_q,next_u

In [ ]:
from sympy import lambdify
def get_next_state_lambda():
    return lambdify((q,u,F1,F2,F3,F4,F5,F6,F7,F8),u_dot_simp)

def get_next_state(curr_q,curr_u,control,curr_t,dt,lamb):
    forces=control(curr_t)
    u_dot_f=lamb(curr_q,curr_u,*forces).flatten()
    next_q=curr_q+curr_u*dt
    next_u=curr_u+u_dot_f*dt
    return next_q,next_u
#test lambdify
qq=np.zeros(6)
uu=np.zeros(6)
lamb=get_next_state_lambda()
lamb(qq,uu,0,0,0,0,1,-1,-1,1)
#lamb(qq,uu,0,0,0,0,1,1,1,1)


In [ ]:
#dump
import dill
dill.settings['recurse'] = True
dill.dump(lamb,open('lambda.pkl','wb'))

In [ ]:
def plot(t,y):
    plt.figure()
    plt.subplot(2,2,1)
    plt.plot(t, y[:, :3])
    plt.legend([latex(s, mode='inline') for s in 'xyz'])
    plt.xlabel('[sec]')
    plt.ylabel('[meters]')
    plt.title('a) Position vs Time')
    plt.subplot(2,2,2)
    plt.plot(t, np.rad2deg(y[:, 3:6]))
    plt.legend([latex(s, mode='inline') for s in 'ypr'])
    plt.xlabel('[sec]')
    plt.ylabel('[deg]')
    plt.title('b) Rotations vs Time')
    plt.subplot(2,2,3)
    plt.xlabel('[meters]')
    plt.ylabel('[meters]')
    plt.title('c) XY plane plot')
    plt.plot(y[:,0],y[:,1])
    plt.axis('equal')
    plt.subplot(2,2,4)
    plt.plot(t, np.rad2deg(y[:, 9:12]))
    plt.legend([latex(s, mode='inline') for s in ['Rz','Ry','Rx']])
    plt.xlabel('[sec]')
    plt.ylabel('[deg/sec]')
    plt.title('d) Angular velocities')
    

In [ ]:
def sim(controller):
    curr_q = np.zeros(6)
    curr_u = np.zeros(6)
    frames_per_sec = 60.0
    final_time = 40.0
    dt=1.0/frames_per_sec
    y=[]
    curr_time=0
    t = np.linspace(0.0, final_time, int(final_time * frames_per_sec))
    lamb=get_next_state_lambda()
    for curr_time in t:
        #next_q,next_u=get_next_state(curr_q,curr_u,controller,curr_time,dt,subs)
        next_q,next_u=get_next_state(curr_q,curr_u,controller,curr_time,dt,lamb)
        next_q,next_u=next_q.flatten(),next_u.flatten()
        y.append(np.hstack((next_q,next_u)))
        curr_q,curr_u=next_q,next_u
        #print(next_u)
    y=np.array(y,dtype='float32')
    return(t,y)


In [ ]:
#only rotation
def controller1(t):
    if t<20:
        return [0.0 for i in range(8)]
    else:
        return [0,0,0,0, 1+0.00,-1+0.00,-1+0.00,1+0.00]
t,y=sim(controller1)
plot(t,y)

In [ ]:
# l ,r f ,b
def controller2(t):
    if t<10: #left
        return [0,0,0,0.0, -1,1,-1,1]
    elif t<20:
        return [0,0,0,0.0, 1,-1,1,-1]
    elif t<30:
        return [0,0,0,0.0, 1,1,1,1]
    else:
        return [0,0,0,0.0, -1,-1,-1,-1]
t,y=sim(controller2)
plot(t,y)

In [ ]:
# d,u
def controller3(t):
    if t<20: #left
        return [1,1,1,1, 0,0,0,0]
    else:
        return [-1,-1,-1,-1 ,0,0,0,0]
t,y=sim(controller3)
plot(t,y)

In [ ]:
# pitch 
def controller4(t):
    if t<20: #left
        return [1,1,-1,-1, 0,0,0,0]
    else:
        return [-1,-1,1,1 ,0,0,0,0]
t,y=sim(controller4)
plot(t,y)

In [ ]:
# roll 
def controller5(t):
    if t<20: #left
        return [1,-1,-1,1, 0,0,0,0]
    else:
        return [-1,1,1,-1 ,0,0,0,0]
t,y=sim(controller5)
plot(t,y)